In [6]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [7]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [8]:
# Using IEX token
# from secrets import IEX_CLOUD_API_TOKEN

ImportError: cannot import name 'IEX_CLOUD_API_TOKEN' from 'secrets' (/opt/anaconda3/lib/python3.7/secrets.py)

In [9]:
# Making API call
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'TG(SOA /NLBSCQMG )DLAKENAATES ELR',
 'calculationPrice': 'close',
 'open': 128.81,
 'openTime': 1671060842907,
 'openSource': 'ocaiiffl',
 'close': 127,
 'closeTime': 1641252915703,
 'closeSource': 'fcliafoi',
 'high': 125,
 'highTime': 1657428605690,
 'highSource': 'eosCl',
 'low': 124,
 'lowTime': 1644743054722,
 'lowSource': 'Celos',
 'latestPrice': 125,
 'latestSource': 'Close',
 'latestTime': 'April 1, 2021',
 'latestUpdate': 1695750383182,
 'latestVolume': 76283039,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 124.254,
 'delayedPriceTime': 1626553643467,
 'oddLotDelayedPrice': 128.41,
 'oddLotDelayedPriceTime': 1622235849062,
 'extendedPrice': 128,
 'extendedChange': 0,
 'extendedChangePercent': 0,
 'extendedPriceTime': 1667606426485,
 'previousClose': 122.26,
 'previousVolume': 121749505,
 'change': 0.89,
 'changePercent': 0.00701,
 'volume': 77830529,
 'iexMark

In [10]:
data['latestPrice']

125

In [11]:
data['marketCap']

2074199952165

## Creating pandas dataframe using stocks data 

In [12]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']

In [13]:
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [14]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,125,2074199952165,N/A


In [15]:
# Pull data for all stocks using loop API call
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [16]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,132.85,39939152822,N/A
1,AAL,24.26,15969744289,N/A
2,AAP,193.01,12580149627,N/A
3,AAPL,126.00,2116565405826,N/A
4,ABBV,110.35,201056931407,N/A
...,...,...,...,...
500,YUM,115.20,33183294654,N/A
501,ZBH,167.00,34899271003,N/A
502,ZBRA,501.99,27271350897,N/A
503,ZION,56.72,9194202787,N/A


### Using Batch API calls

In [17]:
def get_chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [19]:
symbol_groups = list(get_chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [20]:
final_dataframe = pd.DataFrame(columns = my_columns)

In [21]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [22]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,131,40604283590,N/A
1,AAL,23.89,15533400991,N/A
2,AAP,194.82,12426448046,N/A
3,AAPL,124,2136481242083,N/A
4,ABBV,109,194106729491,N/A
...,...,...,...,...
500,YUM,110,33362736977,N/A
501,ZBH,168,33729396409,N/A
502,ZBRA,492.73,27222440902,N/A
503,ZION,56.31,9383220637,N/A


## Calculate number of shares to buy for your portfolio 

In [23]:
portfolio_size = input("Enter the value of your portfolio : ")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio : ")

Enter the value of your portfolio : 100000


In [24]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,131,40604283590,1
1,AAL,23.89,15533400991,8
2,AAP,194.82,12426448046,1
3,AAPL,124,2136481242083,1
4,ABBV,109,194106729491,1
...,...,...,...,...
500,YUM,110,33362736977,1
501,ZBH,168,33729396409,1
502,ZBRA,492.73,27222440902,0
503,ZION,56.31,9383220637,3


In [25]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,131,40604283590,1
1,AAL,23.89,15533400991,8
2,AAP,194.82,12426448046,1
3,AAPL,124,2136481242083,1
4,ABBV,109,194106729491,1
...,...,...,...,...
500,YUM,110,33362736977,1
501,ZBH,168,33729396409,1
502,ZBRA,492.73,27222440902,0
503,ZION,56.31,9383220637,3


## Formatting using xlsx 

In [26]:
#Formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

#  String format for tickers
#  $XX.XX format for stock prices
#  $XX,XXX format for market capitalization
#  Integer format for the number of shares to purchase

In [27]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [28]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [29]:
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )

0

In [30]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [31]:
# Save to excel file
writer.save()